<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Google Cloud Key Management Secrets Engine

You want to maintain root of trust and control of the encryption key lifecycle.

<img src="images/vault-tokenization-1.png">


**Prerequisites:**
* Vault binary

## Setup

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
which vault
vault --version

\
**EXECUTE the following command in SEPARATE window**

```
VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional
```
OR
`vrd1`

In [ ]:
vault status

In [ ]:
unset VAULT_TOKEN
vault login root

##### Notice Initialized (=true) and Sealed (=false) status 

## Enable the Key Management secrets engine

In [ ]:
vault secrets enable gcpkms

In [ ]:
vault write gcpkms/config credentials=@/Users/tio/gcloud/terraform-gcloud1-785d7e769495.json

### Create a Google Cloud KMS key

Let's write a pair of RSA keys to the secret engine


In [ ]:
vault write gcpkms/keys/my-key\
  key_ring=projects/terraform-gcloud1/locations/asia-southeast1/keyRings/tio-keys \
  rotation_period=72h

### Read keys

In [ ]:
vault read gcpkms/key/rsa-2 -format=json | jq   ###not working

---
# Symmetric Encryption/Decryption
Use a Cloud KMS key for symmetric encryption/decryption.  


| Purpose | Supported Algorithms |
|---------|-----------------------|
| encrypt_decrypt | symmetric_encryption |

In [ ]:
vault write gcpkms/keys/my-key3\
  key_ring=projects/terraform-gcloud1/locations/asia-southeast1/keyRings/tio-keys \
  purpose=encrypt_decrypt \
  algorithm=symmetric_encryption

### Encrypt plaintext

In [ ]:
vault write gcpkms/encrypt/my-key3 plaintext="hello world"

### Decrypt ciphertext

In [ ]:
vault write gcpkms/decrypt/my-key3 ciphertext="CiQAtzWXx1sEvCM2rit4nVLWXxKoZUPzmY88eBwW/TcCTEFWzfISNAAsWQBDXVIiu/AnDLZCHtP27AowgAxj1zN9aqlad4UIktnKGi8XNoHHcvwC/OQfCdTCcfc="

**OR**

In [ ]:
vault write -field=plaintext gcpkms/decrypt/my-key2 ciphertext="CiQAUpRiLOW2HHR9NHmMk2JCNUZbgd75fzUqJdkTJqhfdpRZVUwSNABFEwz2bWrzaCskVQYwI+OgxelTe146xmkD3c89Z1D5SCKsHsVlPdQrTckaHS8Hj76I9/A="

### Rotate underlying encryption key

In [ ]:
vault write -f gcpkms/keys/rotate/my-key3

In [ ]:
vault write gcpkms/reencrypt/my-key2 ciphertext="CiQAUpRiLOW2HHR9NHmMk2JCNUZbgd75fzUqJdkTJqhfdpRZVUwSNABFEwz2bWrzaCskVQYwI+OgxelTe146xmkD3c89Z1D5SCKsHsVlPdQrTckaHS8Hj76I9/A="

### Delete key versions older than min_version

In [ ]:
vault write -f gcpkms/keys/config/my-key3 min_version=10

In [ ]:
vault write -f gcpkms/keys/trim/my-key3

### Delete key

In [ ]:
vault delete gcpkms/keys/my-key3

In [ ]:
vault list gcpkms/keys

### Asymmetric Decryption
Use Cloud KMS key for asymmetric decryption.  In this model Google Cloud manages the key ring and exposes the public key via an API endpoint.  The public key is used to encrypt data offline and produce ciphertext.

| Purpose | Supported Algorithms |
|---------|-----------------------|
| asymmetric_decrypt | rsa_decrypt_oaep_2048_sha256 |
| | rsa_decrypt_oaep_3072_sha256 |
| | rsa_decrypt_oaep_4096_sha256 |



In [ ]:
vault write gcpkms/keys/t2-key\
  key_ring=projects/terraform-gcloud1/locations/asia-southeast1/keyRings/tio2-keys \
  purpose=asymmetric_decrypt \
  algorithm=rsa_decrypt_oaep_4096_sha256

### Retrieve the public key from Cloud KMS

In [ ]:
gcloud kms keys versions get-public-key v1 \
    --location asia-southeast1 \
    --keyring projects/terraform-gcloud1/locations/asia-southeast1/keyRings/tio2-keys  \
    --key t2-key \
    --output-file /tmp/mykey.pub

In [ ]:
openssl pkeyutl -in ~/my-secret-file \
    -encrypt -pubin \
    -inkey ~/mykey.pub \
    -pkeyopt rsa_padding_mode:oaep \
    -pkeyopt rsa_oaep_md:sha256 \
    -pkeyopt rsa_mgf1_md:sha256

**Note** the encryption happens offline (outside of Vault).  The encryption is hapenning with a public key. Only Cloud KMS has the corresponding private key.

In [ ]:
vault write gcpkms/decrypt/t2-key key_version=1 ciphertext=CiQAuMv0lTiKjrF43Lgr4

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">